In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import json
import re
import math


def get_integer(alpha):
    while not math.isclose(alpha, round(alpha), rel_tol=1e-9):  # Use a small tolerance
        alpha *= 10
    return int(alpha)



# get_accuracy file and plot

In [19]:
import colorsys
import csv
import math
from tabnanny import check
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from joblib.parallel import method
from numpy import dtype

from algorithm.dynamic_window import Accuracy_workload as workload_dynamic
from algorithm.fixed_window import Accuracy_workload as workload_fixed
import seaborn as sns

# sns.set_style("whitegrid")
sns.set_palette("Paired")
sns.set_context("paper", font_scale=2)

# Set the font size for labels, legends, and titles

plt.figure(figsize=(3.5, 2))
plt.rcParams['font.size'] = 20


def get_integer(alpha):
    while not math.isclose(alpha, round(alpha), rel_tol=1e-9):  # Use a small tolerance
        alpha *= 10
    return int(alpha)


def scale_lightness(rgb, scale_l):
    # convert rgb to hls
    h, l, s = colorsys.rgb_to_hls(*rgb)
    # manipulate h, l, s values and return as rgb
    return colorsys.hls_to_rgb(h, min(1, l * scale_l), s=s)


#  all time:
method_name = "hoeffding_classifier"
data = pd.read_csv('../../../../../../result_' + method_name + '.csv', dtype={"zip_code": str})
print(data["gender"].unique())
date_column = "datetime"
# get distribution of compas_screening_date
data[date_column] = pd.to_datetime(data[date_column])
print(data[date_column].min(), data[date_column].max())
date_time_format = True


monitored_groups = [{"gender": 'M'}, {"gender": 'F'}]
print(data[:5])
alpha = 0.996
threshold = 0.3
label_prediction = "prediction"
label_ground_truth = "rating"
correctness_column = "diff_binary_correctness"
use_two_counters = True


def get_accuracy(T_b, T_in, checking_interval = "1 hour", time_unit = "1 hour", alpha = 0.996):
    import csv 
    print("====================== dynamic workload ==================")
    print("T_b = {}, T_in = {}".format(T_b, T_in))
    (DFMonitor_bit_dynamic, DFMonitor_counter_dynamic, uf_list_dynamic, accuracy_list_dynamic,
     counter_list_correct_dynamic, counter_list_incorrect_dynamic, window_reset_times, check_points) \
    = workload_dynamic.traverse_data_DFMonitor_and_baseline(data, date_column, date_time_format,
                                                    monitored_groups,  threshold, alpha,
                                                    time_unit, checking_interval, label_prediction,
                                                    label_ground_truth, correctness_column, T_b, T_in, use_two_counters)
    
    print(len(uf_list_dynamic))
    
    print(accuracy_list_dynamic[:5])
    
    # save the result to a file
    male_time_decay_dynamic = [x[0] for x in accuracy_list_dynamic]
    # male_time_decay_fixed = [x[0] for x in accuracy_list_fixed]
    female_time_decay_dynamic = [x[1] for x in accuracy_list_dynamic]
    # female_time_decay_fixed = [x[1] for x in accuracy_list_fixed]
    
    filename = f"movielens_compare_Accuracy_{method_name}_gender_dynamic_Tb_{T_b}_Tin_{T_in}_alpha_{str(get_integer(alpha))}_time_unit_{time_unit}_check_interval_{checking_interval}.csv"
    with open(filename, "w", newline='') as csvfile:
        writer = csv.writer(csvfile, delimiter=',')
        writer.writerow(["male_time_decay_dynamic", "female_time_decay_dynamic"])
        for i in range(len(female_time_decay_dynamic)):
            writer.writerow([accuracy_list_dynamic[i][0], accuracy_list_dynamic[i][1]])
    
    
    
    filename = f"movielens_dynamic_window_reset_time_Tb_{T_b}_Tin_{T_in}_time_unit_{time_unit}.csv"
    with open(filename, "w", newline='') as csvfile:
        writer = csv.writer(csvfile, delimiter=',')
        writer.writerow(["dynamic_window_reset_time"])
        for i in range(len(window_reset_times)):
            writer.writerow([window_reset_times[i]])
    
    
    # ################################################## draw the plot #####################################################
    import csv
    
    import matplotlib
    import pandas as pd
    import numpy as np
    import matplotlib.pyplot as plt
    from algorithm.fixed_window import FNR_workload as workload
    import seaborn as sns
    import colorsys
    import colormaps as cmaps
    
    sns.set_palette("Paired")
    sns.set_context("paper", font_scale=2)
    
    plt.figure(figsize=(3.5, 2))
    plt.rcParams['font.size'] = 20
    
    def scale_lightness(rgb, scale_l):
        h, l, s = colorsys.rgb_to_hls(*rgb)
        return colorsys.hls_to_rgb(h, min(1, l * scale_l), s=s)
    
    
    df = pd.read_csv(f"movielens_compare_Accuracy_hoeffding_classifier_gender_dynamic_Tb_{T_b}_Tin_{T_in}_alpha_{str(get_integer(alpha))}_time_unit_{time_unit}_check_interval_{checking_interval}.csv")
    print(df)
    
    x_list = np.arange(0, len(df))
    male_time_decay = df["male_time_decay_dynamic"].tolist()
    female_time_decay = df["female_time_decay_dynamic"].tolist()
    pair_colors = ["blue", "darkorange"]
    
    fig, ax = plt.subplots(figsize=(3.5, 2))
    plt.subplots_adjust(left=0.1, right=0.9, top=0.8, bottom=0.1)
    
    
    ax.plot(x_list, male_time_decay, linewidth=1, markersize=1, label='Male', linestyle='-', marker='o', color=pair_colors[0])
    ax.plot(x_list, female_time_decay, linewidth=1, markersize=1, label='Female', linestyle='-', marker='o', color=pair_colors[1])
    
    
    plt.xticks([], [], rotation=0, fontsize=20)
    plt.ylabel('Accuracy', fontsize=14, labelpad=-1)
    plt.grid(True, axis='y')
    plt.yticks(fontsize=12)
    plt.ylim(0.4, 1.0)
    plt.yticks([0.4, 0.6, 0.8, 1.0], ["0.4", "0.6", "0.8", "1.0"], fontsize=12)
    plt.tight_layout()
    plt.legend(loc='upper left', bbox_to_anchor=(-0.1, 1.3), fontsize=12,
                   ncol=2, labelspacing=0.2, handletextpad=0.2, markerscale=1.5,
                   columnspacing=0.2, borderpad=0.2, frameon=True)
    plt.savefig(f"Acc_hoeffding_time_decay_gender_dynamic_Tb_{T_b}_Tin_{T_in}_alpha_{str(get_integer(alpha))}_time_unit_{time_unit}_check_interval_{checking_interval}.png", bbox_inches='tight')
    plt.show()

['M' 'F']
1997-09-20 05:05:10 1998-04-23 01:10:38
   user  item           timestamp                            title  \
0   259   255  874731910000000000  My Best Friend's Wedding (1997)   
1   259   286  874731927000000000      English Patient, The (1996)   
2   259   298  874731954000000000                  Face/Off (1997)   
3   259   185  874731981000000000                    Psycho (1960)   
4   259   173  874732043000000000       Princess Bride, The (1987)   

         release_date                              genres   age age_2groups  \
0  866764800000000000                     comedy, romance  21.0        7-30   
1  848016000000000000                 drama, romance, war  21.0        7-30   
2  867369600000000000            action, sci_fi, thriller  21.0        7-30   
3 -315619200000000000           horror, romance, thriller  21.0        7-30   
4  536457600000000000  action, adventure, comedy, romance  21.0        7-30   

  age_4groups gender occupation zip_code  rating  pred

<Figure size 350x200 with 0 Axes>

In [20]:


# time_unit = "1 hour"
# T_b = 72
# T_in = 8
# checking_interval = "1 hour"




# 1 hour time unit, alter fixed window size, compare results

In [21]:

def aggregated_mae(lists):
    total_mae = 0
    pair_count = 0
    num_lists = len(lists)
    for i in range(num_lists):
        for j in range(i + 1, num_lists):
            mae = np.mean(np.abs(np.array(lists[i]) - np.array(lists[j])))
            total_mae += mae
            pair_count += 1
    if pair_count > 0:
        average_mae = total_mae / pair_count
        print(f"Average MAE across all pairs: {average_mae}")
        return average_mae
    else:
        return 0



In [23]:
def change_window_size(window_size_unit_list):
        
    time_unit = "1 hour"
    alpha = 0.996
    checking_interval = "1 hour"
    method_name = "hoeffding_classifier"
    data = pd.read_csv('../../../../../../result_' + method_name + '.csv', dtype={"zip_code": str})
    print(data["gender"].unique())
    date_column = "datetime"
    female_lists = []
    male_lists = []
    idx = 0
    
    for window_size_units in window_size_unit_list:
        filename = f"movielens_compare_Accuracy_hoeffding_classifier_gender_per_item_alpha_{str(get_integer(alpha))}_time_unit_{time_unit}*{window_size_units}_check_interval_{checking_interval}.csv"
        df = pd.read_csv(filename)
        male_time_decay = df["male_time_decay"].tolist()
        female_time_decay = df["female_time_decay"].tolist()
        female_lists.append(female_time_decay)
        male_lists.append(male_time_decay)
    avg_mae_male = aggregated_mae(male_lists)
    avg_mae_female = aggregated_mae(female_lists)
    print(f"Average MAE: male {avg_mae_male}, female {avg_mae_female}\n")
    return avg_mae_male, avg_mae_female


window_size_unit_list = ['1', '6', '12', '24', '24*3', '24*7', '24*7*2', '24*30', "24*30*2", '24*30*3']
avg_male_list = []
avg_female_list = []

m, f = change_window_size(window_size_unit_list)
print(m, f)

# print(np.mean(avg_male_list))
# print(np.mean(avg_female_list))

# change_Tb_fix_Tin(2, Tb_list)
# 
# change_Tb_fix_Tin(4, Tb_list)
# change_Tb_fix_Tin(8, Tb_list)
# change_Tb_fix_Tin(12, Tb_list)
# change_Tb_fix_Tin(24, Tb_list)
# change_Tb_fix_Tin(48, Tb_list)  
# change_Tb_fix_Tin(72, Tb_list)
# change_Tb_fix_Tin(96, Tb_list)
# change_Tb_fix_Tin(120, Tb_list)
# change_Tb_fix_Tin(144, Tb_list)
# change_Tb_fix_Tin(168, Tb_list)

['M' 'F']
Average MAE across all pairs: 0.006284418850017413
Average MAE across all pairs: 0.012505857544087538
Average MAE: male 0.006284418850017413, female 0.012505857544087538

0.006284418850017413 0.012505857544087538


In [36]:

print("done")


done


In [ ]:
def change_Tin_fix_Tb(T_b, Tin_list):
        
    time_unit = "1 hour"
    alpha = 0.996
    checking_interval = "1 hour"
    method_name = "hoeffding_classifier"
    data = pd.read_csv('../../../../../result_' + method_name + '.csv', dtype={"zip_code": str})
    print(data["gender"].unique())
    date_column = "datetime"

    idx = 0
    
    for T_in in Tin_list:
        df = pd.read_csv(f"movielens_compare_Accuracy_{method_name}_gender_dynamic_Tb_{T_b}_Tin_{T_in}_alpha_{str(get_integer(alpha))}_time_unit_{time_unit}_check_interval_{checking_interval}.csv")
        x_list = np.arange(0, len(df))
        male_time_decay = df["male_time_decay_dynamic"].tolist()
        female_time_decay = df["female_time_decay_dynamic"].tolist()
        mae = np.mean(np.abs(np.array(female_time_decay) - np.array(male_time_decay)))
        print(mae)
        idx += 1



Tb_list = [2, 4, 6, 8, 12, 24, 48, 72]
T_in = 1
change_Tb_fix_Tin(1, Tb_list)
change_Tb_fix_Tin(2, Tb_list)
change_Tb_fix_Tin(4, Tb_list)